# Offline Engine API

SGLang provides a direct inference engine without the need for an HTTP server, especially for use cases where additional HTTP server adds unnecessary complexity or overhead. Here are two general use cases:

- Offline Batch Inference
- Custom Server on Top of the Engine

This document focuses on the offline batch inference, demonstrating four different inference modes:

- Non-streaming synchronous generation
- Streaming synchronous generation
- Non-streaming asynchronous generation
- Streaming asynchronous generation

Additionally, you can easily build a custom server on top of the SGLang offline engine. A detailed example working in a python script can be found in [custom_server](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/custom_server.py).



## Nest Asyncio
Note that if you want to use **Offline Engine** in ipython or some other nested loop code, you need to add the following code:
```python
import nest_asyncio

nest_asyncio.apply()

```

## Advanced Usage

The engine supports [vlm inference](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/offline_batch_inference_vlm.py) as well as [extracting hidden states](https://github.com/sgl-project/sglang/blob/main/examples/runtime/hidden_states). 

Please see [the examples](https://github.com/sgl-project/sglang/tree/main/examples/runtime/engine) for further use cases.

## Offline Batch Inference

SGLang offline engine supports batch inference with efficient scheduling.

In [1]:
# launch the offline engine
import asyncio

import sglang as sgl
import sglang.test.doc_patch
from sglang.utils import async_stream_and_merge, stream_and_merge

llm = sgl.Engine(model_path="qwen/qwen2.5-0.5b-instruct")

[2025-12-24 02:41:12] INFO utils.py:148: Note: detected 112 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.


[2025-12-24 02:41:12] INFO utils.py:151: Note: NumExpr detected 112 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.


[2025-12-24 02:41:12] INFO utils.py:164: NumExpr defaulting to 16 threads.


[2025-12-24 02:41:15] WARNING server_args.py:1529: Attention backend not specified. Use fa3 backend by default.


[2025-12-24 02:41:15] INFO server_args.py:2405: Set soft_watchdog_timeout since in CI


...

Capturing batches (bs=112 avail_mem=61.19 GB):   5%|▌         | 1/20 [00:00<00:09,  1.97it/s]

Capturing batches (bs=72 avail_mem=61.17 GB):  30%|███       | 6/20 [00:00<00:01,  9.14it/s]

Capturing batches (bs=32 avail_mem=61.15 GB):  60%|██████    | 12/20 [00:01<00:00, 15.96it/s]

Capturing batches (bs=8 avail_mem=61.13 GB):  75%|███████▌  | 15/20 [00:01<00:00, 16.51it/s] 

Capturing batches (bs=1 avail_mem=61.11 GB): 100%|██████████| 20/20 [00:01<00:00, 13.80it/s]


### Non-streaming Synchronous Generation

In [2]:
prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Hello, my name is
Generated text:  Daphne. I'm a young woman who has a lot of experiences. My sister and I both went through a lot of emotional trauma in our lives. One of the most difficult things about being a woman is feeling like you're not enough, no matter what you do. That's because of the way society has treated women for so long. I've had to fight against all kinds of negativity and discrimination. This includes being treated as less desirable than men, because of things like my appearance. I've had to fight for equal rights and equal opportunities. I've had to fight for my own safety and trust. I've had to
Prompt: The president of the United States is
Generated text:  a four-term member of the government of the United States. A four-term president does not run for re-election after two years of office. Which of the following two most likely candidates were the ones who were the candidates in the Republican Party nomination for president of the United States during the

### Streaming Synchronous Generation

In [3]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {
    "temperature": 0.2,
    "top_p": 0.9,
}

print("\n=== Testing synchronous streaming generation with overlap removal ===\n")

for prompt in prompts:
    print(f"Prompt: {prompt}")
    merged_output = stream_and_merge(llm, prompt, sampling_params)
    print("Generated text:", merged_output)
    print()


=== Testing synchronous streaming generation with overlap removal ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is


Generated text:  [Name] and I'm a [occupation] who has been [number of years] in the industry. I'm passionate about [reason for passion], and I'm always looking for ways to [action or achievement]. I'm confident in my abilities and always strive to [action or achievement]. I'm a [character trait or quality] who is [reason for character trait or quality]. I'm [character trait or quality] and I'm [reason for character trait or quality]. I'm [character trait or quality] and I'm [reason for character trait or quality]. I'm [character trait or quality] and I'm [

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is


Generated text:  Paris, also known as the City of Light, and is the largest city in Europe by population. It is located on the Seine River and is the seat of government, administration, and culture for the country. Paris is known for its iconic landmarks such as the Eiffel Tower, Notre-Dame Cathedral, and the Louvre Museum, as well as its rich history and diverse cultural scene. The city is also home to many famous museums, including the Musée d'Orsay, the Musée Rodin, and the Musée d'Orsay. Paris is a major tourist destination and a popular destination for business and leisure

Prompt: Explain possible future trends in artificial intelligence. The future of AI is


Generated text:  likely to be characterized by several key trends:

1. Increased integration with human intelligence: AI is likely to become more integrated with human intelligence, allowing machines to learn from and adapt to human behavior and decision-making processes. This could lead to more sophisticated and nuanced AI systems that can better understand and respond to human emotions and motivations.

2. Greater reliance on machine learning: AI is likely to become more machine learning-driven, with machines being able to learn from data and improve their performance over time. This could lead to more efficient and effective AI systems that can handle a wider range of tasks.

3. Increased focus on ethical and social implications: As



### Non-streaming Asynchronous Generation

In [4]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous batch generation ===")


async def main():
    outputs = await llm.async_generate(prompts, sampling_params)

    for prompt, output in zip(prompts, outputs):
        print(f"\nPrompt: {prompt}")
        print(f"Generated text: {output['text']}")


asyncio.run(main())


=== Testing asynchronous batch generation ===



Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text:  [Name] and I'm a self-proclaimed newbie who's always trying to improve my craft. I'm passionate about designing and creating, and I really enjoy the process of coming up with original ideas and turning them into something truly unique. I'm always on the lookout for new and exciting projects to work on, and I'm eager to learn and improve my skills in this field. What's your name? What's your passion? What's the most exciting thing you've ever done? What's the one piece of advice you have for someone trying to improve their craft? I'm glad to have you on board! Your name is [

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is
Generated text:  Paris. It is the largest city in the country and the economic and cultural center of France. The city is known for its rich history, vibrant arts scene, and numerous attractions. It serves as

### Streaming Asynchronous Generation

In [5]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous streaming generation (no repeats) ===")


async def main():
    for prompt in prompts:
        print(f"\nPrompt: {prompt}")
        print("Generated text: ", end="", flush=True)

        # Replace direct calls to async_generate with our custom overlap-aware version
        async for cleaned_chunk in async_stream_and_merge(llm, prompt, sampling_params):
            print(cleaned_chunk, end="", flush=True)

        print()  # New line after each prompt


asyncio.run(main())


=== Testing asynchronous streaming generation (no repeats) ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text: 

 [

Your

 Name

].

...

 revolution

ize

 industries

 such

 as

In [6]:
llm.shutdown()